# Feature selection methods:

Implement and compare different feature selection methods:

# This implementation features: 
   - Correlation Filter (Pearson) 
   - Mutual information filter
   - wrapper method based on KNN 
   - Base code for an embedding IDEA featuring a NN
   - Comparison functions and metrics

In [147]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import math
import operator
import pickle

In [148]:
# data preperation class 
class data_prep:                                 #Data preparation and normalization for the model 
    def __init__(self,train,test):
        self.data = open(train, "r+")
        self.data = self.data.read()
        self.data = self.data.splitlines()
        self.labels = open(test, "r+")
        self.labels = self.labels.read()
        self.labels = self.labels.splitlines()
        for x in range(len(self.data)):
            self.data[x] = self.data[x].split(" ")
        for x in range(len(self.data)):
            self.data[x] = self.data[x][:-1]
                       
    def transform(self):
        data = self.data
        label = self.labels
        for w in range(len(data)):
            if label[w] == "-1":
                label[w] = 0
            else:
                label[w] = 1
            for y in range(len(data[w])):
                data[w][y] = float(data[w][y]) 
        self.transformedX = np.array(data)
        self.transformedY = np.array(label,ndmin=2).T
            
    def normalize(self):
        features = self.transformedX.T
        for y in range(len(features)):
            maxi = max(features[y])
            mini = min(features[y])
            if mini == maxi: 
                for z in range (len(features[y])):
                    features[y][z] =  None  
            else:
                for z in range (len(features[y])):
                    features[y][z] =  (features[y][z]- mini)/ (maxi - mini)
        self.normalizedX = features.T

In [149]:
train = data_prep("data/FS/arcene_train.data","data/FS/arcene_train.labels")            
train.transform()  
train.normalize()

test = data_prep("data/FS/arcene_valid.data","data/FS/arcene_valid.labels")            
test.transform()      
test.normalize()

In [170]:
class feature_selector:            
    def __init__(self,dataX,dataY,NNMethod):
        self.X = dataX
        self.Y = dataY.T[0]
        self.f = len(self.X.T)    
        self.EY = np.mean(self.Y)
        self.Ydiff = self.Y - self.EY
        self.stdY = np.std(self.Y)
        self.NN = NNMethod
        
    def euclidean(self, basis, example):        # Distance function to calculate 
        eucl = []                               # the distance for all datapoints.
        for v in range(0,len(basis)):
            distance = 0 
            for y in range(len(example)):
                distance = distance + ((basis[v][y]-example[y])**2)
            distance**(1/2)
            eucl.append(distance)
        return np.array(eucl,ndmin=2).T
    
    def accuracy(self,labels,preds):
            return (np.count_nonzero(np.equal(labels,preds))) / len(labels) *100 
    
    def KNN(self,basis,toPredict): 
        predictions = []
        for x in range(len(toPredict)):
            differences = np.array(self.euclidean(basis,toPredict[x])) 
            idx = np.argmin(differences)
            predictions.append(self.Y[idx])
        return predictions
        
    def wrapper_step_forward(self,k,X_test,Y_test):       #wrapper for KNN selection
        currentAccuracy = None
        listing = []
        X_final = False
        X_final_test = False
        X = self.X.T
        X_test = X_test.T
        for x in range(k):
            best = 0
            row = None
            if isinstance(X_final,bool) :
                for y in range(len(X)): 
                    predict = self.KNN(np.expand_dims(X[y].T,axis = 1),np.expand_dims(X_test[y].T,axis = 1))
                    accuracy = self.accuracy(Y_test.T[0],predict)
                    if accuracy > best:
                        best = accuracy
                        row = y
                X_final = np.expand_dims(X[row],axis=1).T
                X_final_test = np.expand_dims(X_test[row],axis=1).T
                X_test = np.delete(X_test,row,0)
                X = np.delete(X,row,0)
                listing.append(row)
                currentAccuracy = best
            else:
                for y in range(len(X)): 
                    data = np.concatenate((X_final, np.expand_dims(X[y],axis = 1).T),axis=0).T
                    test = np.concatenate((X_final_test,np.expand_dims(X_test[y],axis = 1).T),axis=0).T
                    predict = self.KNN(data,test)
                    accuracy = self.accuracy(Y_test.T[0],predict)
                    if accuracy > best:
                        best = accuracy
                        row = y
                if best > currentAccuracy: 
                    X_final = np.concatenate((X_final,np.expand_dims(X[row],axis=1).T),axis=0)
                    X_final_test = np.concatenate((X_final_test,np.expand_dims(X_test[row],axis=1).T),axis=0)
                    X_test = np.delete(X_test,row,0)
                    X = np.delete(X,row,0)
                    listing.append(row)
                    currentAccuracy = best
                else: 
                    break
        print("For features " + str(listing) + " the best accuracy was achieved.  Accuracy: "  + str(self.accuracy(Y_test.T[0],self.KNN(X_final.T,X_final_test.T)))+ "%")
        return  listing  
    
    def cov(self):          
        differenceX = self.X - np.expand_dims(np.mean(self.X,axis = 0),1).T
        return np.sum(differenceX.T * self.Ydiff,axis = 1) /(len(self.Y)-1)
    
    def pearson(self):
        covariance = self.cov()
        std = np.std(self.X.T,axis=1) 
        return covariance / std*self.stdY
    
    def correlation_filter(self,k):        #Feature selection based on Correlation matrix
        pearson = self.pearson()
        pearson = np.where(np.isnan(pearson),0,pearson)  # hack to get rid of NaNs
        selection = np.flip(np.argsort(np.absolute(pearson)))[:k]
        print("Features " + str(selection) + " where selected")
        return selection
    
    def mutual_information(self,where):  # Information based Feature
        distribution = self.X.T[where]
        pY = {}
        pX = {}
        pXY = {}
        if np.isnan(self.X.T[where][0]):
            return 0
        value = 1/len(self.Y)
        for x in self.Y:
            if x in pY: 
                pY[x] += value
            else: 
                pY[x] = value
        for y in distribution:
            if y in pX: 
                pX[y] += value
            else: 
                pX[y] = value
        for z in range(len(self.X)):
            pXY[str(1) + " " +  str(distribution[z])] = 0 
            pXY[str(0) + " " +  str(distribution[z])] = 0 
        for u in range(len(self.X)):
            pXY[str(self.Y[u]) + " " +  str(distribution[u])] +=value
        I = 0       
        for keyY, valueY in pY.items():
            for keyX, valueX in pX.items():
                a = pXY[str(keyY)+ " " + str(keyX)]
                if a == 0:
                    pass
                else:
                    I+= a * math.log(a/(valueY*valueX))
        return I
                        
    def find_max_information(self,k):      
        scores = []
        for x in range(len(self.X[0])):
            scores.append(self.mutual_information(x))
        scores = np.array(scores)
        best = np.flip(np.argsort(scores))[:k]
        print("Features " + str(best) + " where selected")
        return best
    
    def find_best_emdedded(self,k,lr,epochs,jump,Y_test,load):  #Calls NN Method to select Feature subset
        if load == False:
            self.embedded  = self.NN.feature_selection(k,lr,epochs,jump)  #SELECT VERSION (_alt)
        out = []
        for x in range (len(self.embedded)):
            prediction = self.KNN(self.embedded[x][0], self.embedded[x][1])
            out.append(self.accuracy(Y_test.T[0],prediction))
        print("Embedded Method Accuracy: " + str(out))
        return out
            
    def score_selection(self,selection,X_test,Y_test): # Runs KNN wih feature selection
        X = np.take(self.X.T, selection,axis=0).T
        testX = np.take(X_test.T, selection,axis=0).T
        prediction = self.KNN(X,testX)
        return self.accuracy(Y_test.T[0],prediction)
    
    def reconstruct_indices(self,arr): #FIX 
        new = []
        for x in range(len(arr)): 
            element = arr[x]
            check = len([i for i in arr[:x] if i < element])
            new.append(element+check)
        return new
            
    def feature_selection(self,steps,X_test,Y_test,load):  #Scores all methods and compares
        if load == False:
            self.filter1 = self.find_max_information(steps[-1])
            self.filter2 = self.correlation_filter(steps[-1])
            self.wrapper = self.self.wrapper_step_forward(steps[-1],X_test,Y_test)
        embeddedScore = self.find_best_emdedded(steps,0.1,20,1,Y_test,load)
        #embeddedScore.reverse()
        self.wrapper = self.reconstruct_indices(self.wrapper)
        results = np.zeros((4,len(steps)))
        for x in range(len(steps)): 
            results[0][x] = self.score_selection(self.filter1[:steps[x]],X_test,Y_test)    
            results[1][x] = self.score_selection(self.filter2[:steps[x]],X_test,Y_test)
            results[2][x] = self.score_selection(self.wrapper[:steps[x]],X_test,Y_test)
            results[3][x] = embeddedScore[x]
        if load == False: 
            self.save_selections()
        df_cm = pd.DataFrame(results,columns=steps,index=["information","correlation","wrapper","embedded"])
        print(df_cm)
    
    def save_selections(self):  #save your selection or load preselected Features
            pickle_out = open("feature_selection/information.pickle","wb")
            pickle.dump(self.filter1, pickle_out)
            pickle_out.close()
            pickle_out = open("feature_selection/correlation.pickle","wb")
            pickle.dump(self.filter2, pickle_out)
            pickle_out.close()
            pickle_out = open("feature_selection/wrapper.pickle","wb")
            pickle.dump(self.wrapper, pickle_out)
            pickle_out.close()
            pickle_out = open("feature_selection/embedding.pickle","wb")
            pickle.dump(self.embedded, pickle_out)
            pickle_out.close()
           
            
    def load_selections(self):
            pickle_in = open("feature_selection/information.pickle","rb")
            self.filter1 = pickle.load(pickle_in)
            pickle_in = open("feature_selection/correlation.pickle","rb")
            self.filter2 = pickle.load(pickle_in)
            pickle_in = open("feature_selection/wrapper.pickle","rb")
            self.wrapper = pickle.load(pickle_in)
            pickle_in = open("feature_selection/embedding.pickle","rb")
            self.embedded = pickle.load(pickle_in)     

In [177]:
class feature_selector_embedded_NN:
        def __init__(self, NNeurons,X,Y,X_test,Y_test):
            self.shape = NNeurons
            self.X, self.y = np.where(np.isnan(X.T),0,X.T),Y.T[0]
            self.X_test, self.y_test = X_test.T,Y_test.T[0]
            self.input = self.X.shape[0]
            self.output = np.max(self.y)+1
            self.sizeX = self.X.shape[1]
            self.sizeY = self.y.shape[0]
            self.weights,self.bias = self.initialize_NN(NNeurons)
            
        def initialize_NN(self,NN):  #initialize all the weights based on parameters
            weights = {}
            bias = {}
            weights[0] = np.random.normal(0,0.1,(NN[0], self.input))
            bias[0] = np.random.normal(0,0.1,(NN[0], 1))
            for x in range(1,len(NN)):
                weights[x] = np.random.normal(0,0.1,(NN[x], NN[x-1]))
                bias[x] = np.random.normal(0,0.1,(NN[x], 1))
            weights[len(NN)] = np.random.normal(0,0.1,(self.output, NN[len(NN)-1]))
            bias[len(NN)] = np.random.normal(0,0.1,(self.output, 1))
            return weights,bias
        
        def softmax(self,X):  #Activation functions and derivatives
            X = X-(np.amax(X,axis=0))
            return np.divide(np.exp(X),np.sum( np.exp(X), axis=0,keepdims = True))
        
        def relu(self,Z):
            return np.where(Z<0,0,Z)
        
        def relu_dev(self,Z):
            return np.where(Z<0,0,1)
            
        def forward(self,inputs):  #forward step (full data matrics is forwarded at once)
            caches = []
            cache = []
            out = np.dot(self.weights[0],inputs) + self.bias[0]
            cache.append(out)
            out = self.relu(out)
            cache.append(out)
            caches.append(cache)
            for x in range (1,len(self.weights)-1):
                cache = []
                out = np.dot(self.weights[x],out) + self.bias[x]
                cache.append(out)
                out = self.relu(out)
                cache.append(out)
                caches.append(cache)
            out = np.dot(self.weights[len(self.weights)-1],out) + self.bias[len(self.weights)-1]
            cache = []
            cache.append(out)
            out = self.softmax(out)
            cache.append(out)
            caches.append(cache)
            return out, np.array(caches)
        
        def backward(self,cache,lr): #backward step based on full dataset (update included)
            dWs = []
            dBs = []
            y = self.oneHotEncoding(self.y)
            dZ = cache[len(cache)-1][1]- y
            dW = np.dot(dZ, cache[len(cache)-2][1].T)/self.sizeX
            dB = np.sum(dZ, axis=1, keepdims=True)/self.sizeX
            dWs.append( lr * dW)
            dBs.append( lr * dB)
            for x in range (len(cache)-1,1,-1): 
                dZ = np.dot(self.weights[x].T,dZ)*self.relu_dev(cache[x-1][0])
                dW = np.dot(dZ, cache[x-2][1].T)/self.sizeX
                dB = np.sum(dZ, axis=1, keepdims=True)/self.sizeX
                dWs.append( lr * dW)
                dBs.append( lr * dB) 
            dZ = np.dot(self.weights[1].T,dZ)*self.relu_dev(cache[0][0])
            dW = np.dot(dZ, self.X.T)/self.sizeX
            dOut = np.dot(self.weights[0].T,dZ)
            dB = np.sum(dZ, axis=1, keepdims=True)/self.sizeX
            dWs.append( lr * dW)
            dBs.append( lr * dB)
            for y in range(len(dWs)): 
                self.weights[y] = self.weights[y]- dWs[len(dWs)-1-y]
                self.bias[y] = self.bias[y]- dBs[len(dWs)-1-y]
            self.outGrad = dOut
                    
        def oneHotEncoding(self,label): #transform the labels into vectors
            encode = np.zeros((self.output,len(label)))
            for z in range(len(label)):
                encode[self.y[z]][z] = 1
            return encode
        
        def log_loss(self,result): #loss function
            loss = self.oneHotEncoding(self.y)* np.log(result) /self.sizeX
            return -np.sum(loss)
        
        def find_pred(self,probs):   #select the number with the highest probability
            return np.argmax(probs,axis=0)
        
        def confusion_matrix(self,labels,preds): #produce confusion matrix
            confusion = np.zeros((self.output,self.output))
            for x in range(len(preds)):
                confusion[int(labels[x])][int(preds[x])] +=1
            return confusion
        
        def accuracy(self,labels,preds): #accuracy score
            return (np.count_nonzero(np.equal(labels,preds))) / len(labels) *100  
        
        def predict(self,what,heatmap): #predict either the whole train or test data and display
            if what == "train": # the metrics of the predictions
                prob,cache = self.forward(self.X)
                labels = self.oneHotEncoding(self.y)
            elif what == "test":
                prob,cache = self.forward(self.X_test)
                labels = self.oneHotEncoding(self.y_test)
            predictions = self.find_pred(prob)
            if what == "train":
                CM = self.confusion_matrix(self.y,predictions)
                Accuracy = self.accuracy(self.y,predictions)
            if what == "test":
                CM = self.confusion_matrix(self.y_test,predictions)
                Accuracy = self.accuracy(self.y_test,predictions) 
            print("Accuracy with " + str(self.input) + " features: " +  str(Accuracy)+ "%")
            if heatmap == True:
                df_cm = pd.DataFrame(CM)
                plt.figure(figsize = (10,7))
                ax = plt.axes()
                ax.set_title('True Labels = Rows, Predictions = Columms')
                sn.set(font_scale=1)
                sn.heatmap(df_cm, annot=True,cbar = False)
            
        def train(self,lr,iterations):  #Loop over forward and backward as long as you want
            for x in range(iterations):
                result,cache = self.forward(self.X)
                self.backward(cache,lr)
                
        def feature_selection(self,maxFeatures,lr,epochs,k):
            selections = []
            for x in range(0,self.input-maxFeatures[0],k):
                self.train(lr,epochs)
                self.predict("train",False)
                strength =  np.sum(np.abs(self.weights[0]),axis=0)
                worst = np.argsort(strength)[:k]
                self.X = np.delete(self.X,worst,0)
                self.X_test = np.delete(self.X_test,worst,0)
                self.input = self.input - k
                if self.input in maxFeatures: 
                    selections.append([self.X.T, self.X_test.T])
                self.weights,self.bias = self.initialize_NN(self.shape)
            return selections
        
        def feature_selection_alt(self,maxFeatures,lr,epochs,k):
            selections = []
            self.train(lr,epochs)
            self.predict("train",False)
            strength =  np.sum(np.abs(self.weights[0]),axis=0)
            best = np.flip(np.argsort(strength))
            for x in maxFeatures:
                X = np.take(self.X, best[:x],axis=0).T
                X_test = np.take(self.X_test, best[:x],axis=0).T
                selections.append([X, X_test])
            selections.reverse()
            return selections
        
        def feature_selection_alt2(self,maxFeatures,lr,epochs,k):
            selections = []
            self.train(lr,epochs)
            self.predict("train",False)
            strength =  np.sum(np.absolute(self.outGrad),axis=1)
            best = np.flip(np.argsort(strength))
            for x in maxFeatures:
                X = np.take(self.X, best[:x],axis=0).T
                X_test = np.take(self.X_test, best[:x],axis=0).T
                selections.append([X, X_test])
            selections.reverse()
            return selections

In [171]:
# Display preselected Features and their performance

NN = feature_selector_embedded_NN([500],train.transformedX,train.transformedY,test.transformedX,test.transformedY)
selector = feature_selector(train.normalizedX,train.transformedY,NN)
selector.load_selections()
selector.feature_selection([1,2,5],test.transformedX,test.transformedY,True)

Embedded Method Accuracy: [56.00000000000001, 64.0, 63.0]
                1     2     5
information  49.0  51.0  68.0
correlation  63.0  62.0  58.0
wrapper      77.0  81.0  89.0
embedded     56.0  64.0  63.0


In [ ]:
# Run all methods and display result

NN = feature_selector_embedded_NN([500],train.transformedX,train.transformedY,test.transformedX,test.transformedY)
selector = feature_selector(train.normalizedX,train.transformedY,NN)
selector.feature_selection([1,2,5],test.transformedX,test.transformedY,False)

In [178]:
#TestStation for Embedding method

NN = feature_selector_embedded_NN([100],train.transformedX,train.transformedY,test.transformedX,test.transformedY)
selector = feature_selector(train.normalizedX,train.transformedY,NN)
x = selector.NN.feature_selection([10],0.1,20,1)
selector.score_selection(x,test.transformedX,test.transformedY)

Accuracy with 10000 features: 97.0%
Accuracy with 9999 features: 92.0%
Accuracy with 9998 features: 88.0%
Accuracy with 9997 features: 87.0%
Accuracy with 9996 features: 97.0%
Accuracy with 9995 features: 95.0%
Accuracy with 9994 features: 96.0%
Accuracy with 9993 features: 79.0%
Accuracy with 9992 features: 91.0%
Accuracy with 9991 features: 93.0%
Accuracy with 9990 features: 96.0%
Accuracy with 9989 features: 96.0%
Accuracy with 9988 features: 79.0%
Accuracy with 9987 features: 92.0%
Accuracy with 9986 features: 98.0%
Accuracy with 9985 features: 99.0%
Accuracy with 9984 features: 95.0%
Accuracy with 9983 features: 96.0%
Accuracy with 9982 features: 94.0%
Accuracy with 9981 features: 90.0%
Accuracy with 9980 features: 98.0%
Accuracy with 9979 features: 81.0%
Accuracy with 9978 features: 92.0%
Accuracy with 9977 features: 99.0%
Accuracy with 9976 features: 91.0%
Accuracy with 9975 features: 80.0%
Accuracy with 9974 features: 46.0%
Accuracy with 9973 features: 95.0%
Accuracy with 9972 

KeyboardInterrupt: 